# Laboratorio 3: Planning con Qwen3-8B

**Modelo:** Qwen3-8B (4-bit quantized)  
**Estrategia:** Few-Shot Chain-of-Thought (CoT) implícito

### Arquitectura de Prompts: Few-Shot CoT
El `scenario_context` ya contiene K ejemplos resueltos que actúan como demostraciones CoT:
el modelo observa el patrón (estado -> goal -> acciones) y lo replica para el último task.
Esta es la estrategia CoT implícita más estable para este formato de datos.

### Dos dominios presentes en los datos
| Dominio | Acciones NL generadas | Notación formal esperada |
|---|---|---|
| **Blocks** | `pick up the X block` | `(engage_payload X)` |
| **Blocks** | `put down the X block` | `(release_payload X)` |
| **Blocks** | `mount_node the X block on top of the Y block` | `(mount_node X Y)` |
| **Blocks** | `unmount_node the X block from on top of the Y block` | `(unmount_node X Y)` |
| **Logic** | `attack object X` | `(attack X)` |
| **Logic** | `succumb object X` | `(succumb X)` |
| **Logic** | `feast object X from object Y` | `(feast X Y)` |
| **Logic** | `overcome object X from object Y` | `(overcome X Y)` |

## Paso 1: Instalacion de dependencias

Se instala `bitsandbytes>=0.46.1` (requerido para cuantizacion 4-bit) junto con
`transformers`, `accelerate` y `einops`. Si `bitsandbytes` no carga correctamente
despues de instalarse, el kernel se reinicia automaticamente para que los cambios
tomen efecto.

In [1]:
import subprocess, sys, os

# Instala/actualiza dependencias necesarias
print("Instalando dependencias...")
subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", "--upgrade",
     "transformers", "accelerate", "einops"],
    check=True
)
subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", "-U", "bitsandbytes>=0.46.1"],
    check=True
)

# Verificamos que bitsandbytes cargo
try:
    import importlib, bitsandbytes
    importlib.reload(bitsandbytes)
    print(f"bitsandbytes {bitsandbytes.__version__} listo.")
except Exception as e:
    print(f"Reiniciando runtime para aplicar bitsandbytes ({e})...")
    os.kill(os.getpid(), 9)

Instalando dependencias...
bitsandbytes 0.49.2 listo.


## Paso 2: Imports y carga del modelo Qwen3-8B

Se carga el modelo con cuantizacion NF4 de 4 bits para reducir el uso de VRAM
y permitir ejecutarlo en la GPU de Colab. `device_map='auto'` distribuye
automaticamente las capas entre GPU y CPU segun la memoria disponible.

In [2]:
import json
import re
import torch
import time
from collections import defaultdict, Counter
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import files

MODEL_ID = "Qwen/Qwen3-8B"

# Configuracion de cuantizacion 4-bit NF4
# NF4 (Normal Float 4) minimiza el error de cuantizacion para pesos con
# distribucion normal, que es el caso tipico de modelos tipo transformer
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

print("Cargando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
    trust_remote_code=True
)

print("Cargando modelo")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",         # distribuye capas automaticamente en GPU/CPU
    torch_dtype=torch.float16,
    trust_remote_code=True
)

model.eval()
print("Modelo listo.")

Cargando tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Modelo listo.


## Paso 3: Cargar Examples.json y Task.json

- `Examples.json`: contiene escenarios ya resueltos con sus secuencias de acciones
  optimas. Se usan como few-shot y para calibrar `complexity_level`.
- `Task.json`: dataset de evaluacion. Solo contiene los escenarios sin resolver.
  El codigo genera el JSON de salida con las soluciones.

In [3]:
uploaded = files.upload()  # Examples.json y Task.json

with open('Examples.json', 'r') as f:
    examples = json.load(f)

with open('Task.json', 'r') as f:
    tasks = json.load(f)

print(f'Examples: {len(examples)}')
print(f'Tasks:    {len(tasks)}')

# Contamos cuantos tasks pertenecen a cada dominio
block_tasks = sum(1 for t in tasks if 'block' in t['scenario_context'])
logic_tasks  = sum(1 for t in tasks if 'block' not in t['scenario_context'])
print(f'\n   Dominio Blocks: {block_tasks} tasks')
print(f'   Dominio Logic:  {logic_tasks} tasks')

Saving Examples.json to Examples.json
Saving Task.json to Task.json
Examples: 574
Tasks:    50

   Dominio Blocks: 31 tasks
   Dominio Logic:  19 tasks


## Paso 4: Calibracion de complexity_level desde Examples.json

En lugar de usar umbrales arbitrarios, se aprenden los rangos reales de acciones
por nivel de complejidad directamente desde los ejemplos etiquetados.

Para cada `complexity_level` presente en `Examples.json`, se calcula el rango
`[min_acciones, max_acciones]` y se usa el maximo como umbral superior del clasificador.
Esto garantiza que los niveles asignados en la salida sean coherentes con los datos reales.

In [4]:
def build_complexity_thresholds(examples: list) -> dict:
    """
    Aprende los umbrales de complexity_level desde Examples.json.
    Para cada nivel presente en los ejemplos, calcula cuantas acciones
    tiene como maximo. Devuelve {level: max_actions} para usarlo como
    clasificador por umbral superior.
    Args:
        examples: lista de ejemplos cargados desde Examples.json
    Returns:
        dict con {nivel_complejidad: maximo_acciones}
    """
    bins = defaultdict(list)
    for ex in examples:
        bins[ex['complexity_level']].append(len(ex['target_action_sequence']))

    # el umbral de cada nivel es el maximo de acciones observado para ese nivel
    thresholds = {lvl: max(counts) for lvl, counts in bins.items()}

    print("Umbrales calibrados desde Examples.json:")
    for lvl in sorted(thresholds):
        print(f"   complexity_level={lvl}: {min(bins[lvl])}-{thresholds[lvl]} acciones")

    return thresholds


# Construimos y guardamos los umbrales globales
COMPLEXITY_THRESHOLDS = build_complexity_thresholds(examples)


def estimate_complexity(num_actions: int, thresholds: dict) -> int:
    """
    Asignamos complexity_level segun el numero de acciones generadas,
    usando los umbrales aprendidos desde Examples.json.
    Recorremos los niveles de menor a mayor y devuelve el primero cuyo
    umbral maximo sea >= num_actions. Si supera todos los rangos,
    devuelve el nivel maximo disponible.
    """
    for level in sorted(thresholds.keys()):
        if num_actions <= thresholds[level]:
            return level
    return max(thresholds.keys())

Umbrales calibrados desde Examples.json:
   complexity_level=2: 2-2 acciones
   complexity_level=4: 4-4 acciones
   complexity_level=6: 6-6 acciones


## Paso 5: Deteccion de dominio y parser NL -> formal

El modelo genera el plan en lenguaje natural (igual que los ejemplos del contexto).
El parser convierte cada linea a notacion formal `(accion args)`.

Se detiene el parseo ante:
- Marcadores de fin: `[PLAN END]`, `[END]`, `[STATEMENT]`
- CoT leaks de Qwen3: frases como `Wait`, `But`, `Let me`, etc.
- Lineas de comentario que empiezan con `#`

In [5]:
def detect_domain(scenario_context: str) -> str:
    """
    Detectamos el dominio del problema a partir del contexto.
    Devuelve 'blocks' si el contexto menciona bloques, 'logic' en caso contrario.
    """
    return 'blocks' if 'block' in scenario_context else 'logic'


def nl_to_formal_blocks(line: str):
    """
    Convierte una linea de plan en lenguaje natural (dominio Blocks)
    a notacion formal. Retorna la accion formal o None si no coincide.
    Mapeo:
        pick up the X block             ->  (engage_payload X)
        put down the X block            ->  (release_payload X)
        mount_node/stack X on top of Y  ->  (mount_node X Y)
        unmount_node/unstack X from Y   ->  (unmount_node X Y)
    """
    line = line.strip().lower()

    m = re.match(r'pick up the (\w+) block', line)
    if m: return f'(engage_payload {m.group(1)})'

    m = re.match(r'put down the (\w+) block', line)
    if m: return f'(release_payload {m.group(1)})'

    # Acepta tanto 'mount_node' como 'stack' como sinonimos
    m = re.match(r'(?:mount_node|stack) the (\w+) block on top of the (\w+) block', line)
    if m: return f'(mount_node {m.group(1)} {m.group(2)})'

    # Acepta tanto 'unmount_node' como 'unstack', con o sin 'on top of'
    m = re.match(r'(?:unmount_node|unstack) the (\w+) block from (?:on top of )?the (\w+) block', line)
    if m: return f'(unmount_node {m.group(1)} {m.group(2)})'

    return None


def nl_to_formal_logic(line: str):
    """
    Convierte una linea de plan en lenguaje natural (dominio Logic)
    a notacion formal. Retorna la accion formal o None si no coincide.
    Mapeo:
        attack object X              ->  (attack X)
        succumb object X             ->  (succumb X)
        feast object X from object Y ->  (feast X Y)
        overcome object X from object Y -> (overcome X Y)
    """
    line = line.strip().lower()

    m = re.match(r'attack object (\w+)', line)
    if m: return f'(attack {m.group(1)})'

    m = re.match(r'succumb object (\w+)', line)
    if m: return f'(succumb {m.group(1)})'

    m = re.match(r'feast object (\w+) from object (\w+)', line)
    if m: return f'(feast {m.group(1)} {m.group(2)})'

    m = re.match(r'overcome object (\w+) from object (\w+)', line)
    if m: return f'(overcome {m.group(1)} {m.group(2)})'

    return None


def parse_plan(raw_text: str, domain: str) -> list:
    """
    Extrae y convierte el plan del texto generado por el modelo.
    Recorre el texto linea por linea y aplica el convertidor NL->formal
    segun el dominio. Detiene el parseo ante:
    - Marcadores de fin: [PLAN END], [END], [STATEMENT]
    - CoT leaks de Qwen3 (razonamiento en voz alta)
    - Lineas de comentario que empiezan con '#'

    Args:
        raw_text: texto generado por el modelo despues de [PLAN]
        domain: 'blocks' o 'logic'
    Returns:
        lista de acciones en notacion formal
    """
    converter = nl_to_formal_blocks if domain == 'blocks' else nl_to_formal_logic
    actions = []

    for line in raw_text.split('\n'):
        line_clean = line.strip()

        # Se detiene ante marcadores de fin de plan
        if any(marker in line_clean.upper() for marker in ['[PLAN END]', '[END]', '[STATEMENT]']):
            break

        # Se detiene ante razonamiento en voz alta (CoT leak de Qwen3)
        if line_clean.lower().startswith(('wait', 'but ', 'note ', 'let me', 'hmm', 'actually', '#')):
            break

        formal = converter(line_clean)
        if formal:
            actions.append(formal)

    return actions

## Paso 6: Funcion de inferencia con Qwen3-8B (temperature=0.0)

### Estrategia: Few-Shot Chain-of-Thought (CoT) implícito

El `scenario_context` tiene la siguiente estructura:
```
[STATEMENT] <estado inicial>    -+
[GOAL] <objetivo>                +- Ejemplo 1 resuelto (few-shot)
[PLAN] accion1 / accion2 ...    -+
...
[STATEMENT] <estado inicial>    -+
[GOAL] <objetivo>                +- Task a resolver
[PLAN]                          -+  <- el modelo completa desde aqui
```

Los ejemplos resueltos codifican el razonamiento CoT implicitamente:
el modelo aprende el patron (estado -> goal -> acciones) y lo replica.

**Parametros de inferencia:**
- `temperature=0.0` + `do_sample=False`: greedy decoding determinista
- `stop_strings=['[PLAN END]']`: detiene la generacion en el marcador de fin
- `repetition_penalty=1.1`: evita loops de repeticion

In [6]:
@torch.inference_mode()
def solve_task(scenario_context: str, max_new_tokens: int = 150) -> dict:
    """
    Resolvemos un task de planificacion con Qwen3-8B usando Few-Shot CoT.
    El scenario_context contiene K ejemplos resueltos que guian al modelo
    implicitamente (few-shot CoT). El modelo completa el ultimo [PLAN].
    Args:
        scenario_context: texto con ejemplos resueltos y el task a resolver
        max_new_tokens: maximo de tokens a generar (150 cubre hasta ~12 acciones)
    Returns:
        dict con 'target_action_sequence', 'complexity_level', 'domain', '_raw'
    """
    domain = detect_domain(scenario_context)

    # Recortamos el prompt exactamente hasta el ultimo [PLAN]
    # El modelo completara desde ese punto en adelante
    prompt = scenario_context.rstrip()
    idx = prompt.rfind('[PLAN]')
    if idx != -1:
        prompt = prompt[:idx + len('[PLAN]')]
    prompt += '\n'

    # Tokenizar el prompt (sin chat template para preservar el formato few-shot)
    inputs = tokenizer(
        prompt,
        return_tensors='pt',
        truncation=True,
        max_length=3072
    ).to(model.device)

    # --- Inferencia determinista ---
    # temperature=0.0 es obligatorio para que los resultados sean reproducibles y auditables.
    # Con do_sample=False se usa greedy decoding, que siempre elige el token con mayor probabilidad ->
    # salida identica en cada ejecucion.

    out_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,          # greedy decoding
        temperature=0.0,          # temperatura explicita = 0 (requerido por enunciado)
        repetition_penalty=1.1,   # penaliza repetir los mismos tokens, evita loops
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        stop_strings=['[PLAN END]'],  # detiene la generacion en el marcador de fin
        tokenizer=tokenizer,          # requerido para que stop_strings funcione
    )

    # Decodificar solo los tokens nuevos (excluir el prompt de entrada)
    gen_ids   = out_ids[0, inputs.input_ids.shape[1]:]
    generated = tokenizer.decode(gen_ids, skip_special_tokens=True)

    # Parsear el texto generado a notacion formal
    actions    = parse_plan(generated, domain)
    complexity = estimate_complexity(len(actions), COMPLEXITY_THRESHOLDS)

    return {
        'target_action_sequence': actions,
        'complexity_level': complexity,
        'domain': domain,
        '_raw': generated  # raw
    }

## Paso 7: Test rapido con un ejemplo conocido

Antes de procesar todos los tasks, se verifica que el pipeline completo
funciona correctamente usando un ejemplo de `Examples.json` cuya respuesta
correcta ya conocemos.

In [16]:
# Usa el ejemplo con indice 2 (ni el primero ni el ultimo para mayor representatividad)
test_ex = examples[2]

print(f'Task ID:    {test_ex["assembly_task_id"]}')
print(f'Dominio:    {detect_domain(test_ex["scenario_context"])}')
print(f'Expected:   {test_ex["target_action_sequence"]}')

# Ejecuta el pipeline completo
result = solve_task(test_ex['scenario_context'])

print(f'Generated:  {result["target_action_sequence"]}')
print(f'Complexity: {result["complexity_level"]}')
print(f'Raw output: {result["_raw"][:200]!r}')

# Verifica match exacto con la solucion esperada
match = result['target_action_sequence'] == test_ex['target_action_sequence']
print(f'\nMatch exacto: {match}')

Task ID:    task_0d236ad4c6
Dominio:    logic
Expected:   ['(feast a c)', '(succumb a)', '(feast c b)', '(overcome c a)', '(attack b)', '(overcome b c)']
Generated:  ['(attack c)', '(overcome c a)', '(attack b)', '(overcome b c)']
Complexity: 4
Raw output: 'attack object c\novercome object c from object a\nattack object b\novercome object b from object c\n[PLAN END]\n\n'

Match exacto: False


## Paso 8: Procesar todos los tasks

Se itera sobre todos los tasks de `Task.json` y se genera la solucion para cada uno.
El tiempo total debe ser menor a 2 minutos segun el enunciado.
Se imprime el progreso en tiempo real incluyendo estado, dominio, nivel de complejidad
y tiempo por task.

In [8]:
results = []
total = len(tasks)
start_global = time.time()

for i, task in enumerate(tasks):
    t0 = time.time()

    # Resuelve el task con el pipeline completo
    solution = solve_task(task['scenario_context'])

    elapsed       = time.time() - t0
    total_elapsed = time.time() - start_global

    # Guarda solo los campos requeridos en la salida
    results.append({
        'assembly_task_id': task['assembly_task_id'],
        'complexity_level': solution['complexity_level'],
        'target_action_sequence': solution['target_action_sequence'],
    })

    # Mostramos el progreso en tiempo real
    status = 'ok   ' if solution['target_action_sequence'] else 'EMPTY'
    print(
        f"{status} [{i+1:02d}/{total}] {task['assembly_task_id']} | "
        f"{solution['domain']:6s} | L{solution['complexity_level']} | "
        f"{len(solution['target_action_sequence'])} acciones | "
        f"{elapsed:.1f}s (total {total_elapsed:.0f}s)"
    )
    if solution['target_action_sequence']:
        print(f"        -> {solution['target_action_sequence']}")
    else:
        # Si no se parseo ninguna accion, mostrar el texto crudo para debugging
        print(f"        Raw: {solution['_raw'][:120].strip()!r}")

print(f'\nProcesados {len(results)}/{total} tasks en {time.time()-start_global:.1f}s')

ok    [01/50] task_f6c3f52f55 | blocks | L6 | 11 acciones | 11.3s (total 11s)
        -> ['(unmount_node orange red)', '(release_payload orange)', '(unmount_node red blue)', '(release_payload red)', '(release_payload blue)', '(engage_payload yellow)', '(mount_node yellow blue)', '(engage_payload red)', '(mount_node red blue)', '(engage_payload orange)', '(mount_node orange red)']
ok    [02/50] task_07a18910c7 | blocks | L6 | 9 acciones | 10.0s (total 21s)
        -> ['(unmount_node red yellow)', '(release_payload red)', '(engage_payload orange)', '(mount_node orange red)', '(unmount_node blue orange)', '(release_payload blue)', '(engage_payload yellow)', '(mount_node yellow orange)', '(engage_payload blue)']
ok    [03/50] task_cbe2649f6b | blocks | L2 | 2 acciones | 3.4s (total 25s)
        -> ['(engage_payload blue)', '(mount_node blue red)']
ok    [04/50] task_4f181b1e7e | blocks | L2 | 2 acciones | 4.7s (total 29s)
        -> ['(unmount_node yellow red)', '(mount_node yellow blue)']

## Paso 9: Estadisticas de resultados

Se muestra un resumen de los resultados para verificar la calidad de las predicciones
antes de generar el archivo de salida.

In [9]:
# Calcula estadisticas de los resultados generados
c_dist  = Counter(r['complexity_level'] for r in results)
empty   = [r for r in results if not r['target_action_sequence']]
avg_act = sum(len(r['target_action_sequence']) for r in results) / len(results)

print('Estadisticas:')
print(f'   Total tasks procesados : {len(results)}')
print(f'   Promedio acciones/plan : {avg_act:.1f}')
print(f'   Distribucion complexity: {dict(sorted(c_dist.items()))}')
print(f'   Tasks sin acciones     : {len(empty)}')

# Lista tasks vacios para revision manual si los hay
if empty:
    print('\n  Tasks vacios (revisar):')
    for r in empty:
        print(f'   {r["assembly_task_id"]}')

print('\n Muestra (primeros 5):')
for r in results[:5]:
    print(f'  {r["assembly_task_id"]} | L{r["complexity_level"]} | {r["target_action_sequence"]}')

Estadisticas:
   Total tasks procesados : 50
   Promedio acciones/plan : 6.9
   Distribucion complexity: {2: 5, 4: 7, 6: 38}
   Tasks sin acciones     : 0

 Muestra (primeros 5):
  task_f6c3f52f55 | L6 | ['(unmount_node orange red)', '(release_payload orange)', '(unmount_node red blue)', '(release_payload red)', '(release_payload blue)', '(engage_payload yellow)', '(mount_node yellow blue)', '(engage_payload red)', '(mount_node red blue)', '(engage_payload orange)', '(mount_node orange red)']
  task_07a18910c7 | L6 | ['(unmount_node red yellow)', '(release_payload red)', '(engage_payload orange)', '(mount_node orange red)', '(unmount_node blue orange)', '(release_payload blue)', '(engage_payload yellow)', '(mount_node yellow orange)', '(engage_payload blue)']
  task_cbe2649f6b | L2 | ['(engage_payload blue)', '(mount_node blue red)']
  task_4f181b1e7e | L2 | ['(unmount_node yellow red)', '(mount_node yellow blue)']
  task_9f39e7f413 | L6 | ['(unmount_node white yellow)', '(release_payl

## Paso 10: Guardar y descargar el JSON de salida

Se genera `output_planning.json` con el formato requerido por el enunciado:
una lista de objetos con `assembly_task_id`, `complexity_level` y `target_action_sequence`.

In [10]:
output_path = 'output_planning.json'

with open(output_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f'Guardado: {output_path}')
print(f'   {len(results)} registros | {sum(len(r["target_action_sequence"]) for r in results)} acciones totales')

print('\nEjemplo primer item:')
print(json.dumps(results[0], indent=2))

files.download(output_path)

Guardado: output_planning.json
   50 registros | 344 acciones totales

Ejemplo primer item:
{
  "assembly_task_id": "task_f6c3f52f55",
  "complexity_level": 6,
  "target_action_sequence": [
    "(unmount_node orange red)",
    "(release_payload orange)",
    "(unmount_node red blue)",
    "(release_payload red)",
    "(release_payload blue)",
    "(engage_payload yellow)",
    "(mount_node yellow blue)",
    "(engage_payload red)",
    "(mount_node red blue)",
    "(engage_payload orange)",
    "(mount_node orange red)"
  ]
}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
## Notas tecnicas

### Arquitectura de Prompts: Few-Shot Chain-of-Thought (CoT) implícito

El prompt tiene esta estructura:
```
[STATEMENT] <estado inicial>    -+
[GOAL] <objetivo>                +- Ejemplo 1 resuelto (few-shot)
[PLAN] accion1 / accion2 ...    -+
...
[STATEMENT] <estado inicial>    -+
[GOAL] <objetivo>                +- Task a resolver
[PLAN]                          -+  <- el modelo completa desde aqui
```

Los ejemplos resueltos actuan como demostraciones CoT: el modelo aprende
el razonamiento (estado -> goal -> secuencia de acciones) implicitamente y
lo replica para el task final. Esta estrategia es mejor que inyectar
scaffolds de texto porque preserva exactamente el formato de salida esperado.

### Calibracion de complexity_level
Aprendida automaticamente desde `Examples.json`. Para cada nivel se calcula
el rango real (min/max acciones) y se usa como clasificador por umbral superior.

### Parametros de generacion
```python
do_sample=False          # greedy decoding
temperature=0.0          # temperatura explicita = 0 (obligatorio segun enunciado)
repetition_penalty=1.1   # evita loops de repeticion
stop_strings=['[PLAN END]']  # corta el output en el marcador de fin
max_new_tokens=150       # suficiente para planes de hasta ~12 acciones
```

### Cumplimiento de requisitos
| Requisito | Estado |
|---|---|
| Modelo Qwen3-8B sin finetunear | OK |
| temperature=0.0 determinista | OK - explicito en model.generate() |
| Arquitectura de prompts (Few-Shot CoT) | OK |
| complexity_level calibrado | OK - aprendido desde Examples.json |
| Formato de salida correcto | OK |
| Tiempo menor a 2 minutos | OK - single-pass greedy, max_new_tokens=150 |